*Remarque liminaire: Python conduit a des résultats identiques à ceux de R, moins complets pour leur interprétation, mais plus rapidement. Il s'agit des principales différences entre R pour "statisticien" et python pour "informaticien", on perd en interprétabilité mais on gagne en vitesse d'exécution.*

# Analyse en composantes principales et classification


#### Accès rapide aux différentes sections:
* [1 Prétraitement des données](#pretraitement)
* [2 Exploration des données](#exploration)
* [3 Analyse en composantes principales](#acp)
* [4 Classification](#cah)



## Introduction

L'objectif de ce TP est de mettre en oeuvre l'ACP et la CAH sur des données économiques, provenant de la base de données "Air Transport Data", gérée maintenant par France Aviation Civile Service.
La première partie consiste à pré-traiter les données (gestion du type des variables, des données manquantes) et à faire de l'exploration des données (statistique descriptive). La deuxième partie consiste à faire une ACP et à l'interpréter. La troisième partie consiste à faire de la classification.

### Description des données

Elles sont décrites par les variables suivantes:

| Variables      | |    Variables (suite) |
| :------------- |-------- | :------------- | 
 Year 	| | Tonne-Kilometres Performed (thousands)	
Airline | | Passenger Load Factor (%)
Code 	| | Period Fin.	
Al Code ICAO 	 || Currency (000)	
    Country 	| | Operating Revenue	
Country Code OAG 	| | Operating Revenue Pax (000)	
Country Code IATA 	 |  | Operating Expenses	
IATA Regions 	| | Operating Profit	
Passengers Carried (number) 	| | Net Result	
Kilometres Flown (thousands) 	| | Employees	
Aircraft Departures (number)	| | Traf. Source	
Hours Flown (number)	| | Fin Source	
Passenger-Kilometres Flown (thousands)	| | Pers. Source	
Available Seat-Kilometres (thousands)	| | Year End










### Importation des données

In [ ]:
# Classe pour la mise en forme de l'affichage de certaines sorties
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
couleur_plot=["grey", "blue","green","cyan","magenta","yellow","red","black"]

import pandas as pd
import numpy as np
# Lecture des données
path = ""
airlines = pd.read_csv(path + "Airlines20062015NA.csv", sep = ";", header = 0)
#airlines = pd.read_csv(path + "AnaDo_JeuDonnees_TemperatFrance.csv", sep = ";", header = 0, encoding='latin-1')
#airlines_df=airlines.set_index('Villes')
# Nombre d'observations
n = airlines.shape[0]
# Nombre de variables
p = airlines.shape[1]
# Vérification du contenu
airlines.head()

<hr style="height:2px" />

## 1 Pré-traitement des données <a class="anchor" id="pretraitement"></a>

Même si ces données sont relativement bien formatées, il faut les explorer pour pouvoir corriger d'éventuels problèmes d'importation, comme le type de données et les données manquantes.

**Q** Combien y a-t-il d'individus? de variables?



### 1.1 Typage des variables

In [ ]:
print(airlines.dtypes)
#airlines.describe()
#airlines.describe().shape[1]

**Q** Combien y a-t-il de variables quantitatives ? qualitatives ?

**Q** Quel est le problème avec les variables qualitatives? Et plus particulièrement avec la variable "OAG"?

In [ ]:
#airlines["Country Code OAG"] = pd.Categorical(airlines["Country Code OAG"], ordered = False)
airlines['Country Code OAG'] = airlines['Country Code OAG'].astype('object')
# convert the 'Date' column to datetime format
airlines['Year End'] = airlines['Year End'].astype('object')
print(airlines.dtypes)

# Vérification du contenu
airlines.head()

### 1.2 Gestion des valeurs manquantes

#### 1.2.1 Détection des valeurs manquantes

In [ ]:
# Y a-t-il des valeurs manquantes
print(airlines.isnull().values.any())
print()
# Nombre de valeurs manquantes pour chaque variable
print(airlines.isnull().sum())
print()
# Nombre totale de valeurs manquantes dans le dataframe
print("Nombre total de valeurs manquantes:",airlines.isnull().sum().sum())

**Q** Commenter les résultats précédents. Y a-t-il des variables que l'on peut éliminer de l'étude?


#### 1.2.2 Gestion des valeurs manquantes par suppression d'individus ou de variables

In [ ]:
# Taille de la dataframe de départ
print("Dimension du jeu de données", airlines.shape)

# Supprimer les individus ou lignes qui possèdent des valeurs manquantes
airlines0 = airlines.dropna(axis = 0)
print("Dimensions sans les individus avec données manquantes", airlines0.shape)

# Supprimer les variables ou colonnes qui possèdent des valeurs manquantes
airlines1 = airlines.dropna(axis = 1)
print("Dimensions sans les variables avec données manquantes", airlines1.shape)

# Supprimer les colonnes dont le nombre de valeurs manquantes est supérieur à 50%
airlines2 = airlines.loc[:, airlines.isnull().sum() < (0.5*n)]
print('Dimensions sans les colonnes ayant + de 50% de données manquantes: ',airlines2.shape)

#### 1.2.3 Gestion des valeurs manquantes par imputation

In [ ]:
# Imputation par la moyenne
airlines3 = airlines.fillna(airlines.mean())
print("Dimensions avec imputation des données manquantes", airlines3.shape)

# Imputation par la médiane
airlines4 = airlines.fillna(airlines.median())
print("Dimensions avec imputation des données manquantes", airlines4.shape)


**Q** Parmi les 5 jeux de données précédents (airlines0-4), le(s)quel(s) choisiriez-vous pour la suite de l'étude? Expliquer pourquoi.

<hr style="height:2px" />

## 2 Exploration des données <a class="anchor" id="exploration"></a>

Pour la suite, on décide de ne travailler qu'avec les années 2015.

In [ ]:
airlines15 = airlines.query('Year == 2015')

airlines2015 = airlines15.set_index('Airline')
del airlines2015['Year']
airlines2015.head()

In [ ]:
print(airlines2015.isnull().sum())

### 2.1 Statistique descriptive univariée

#### 2.1.1 Statistique quantitative

In [ ]:
airlines2015.describe()

In [ ]:
import matplotlib.pyplot as plt

# Histogramme de la variable 'Operating Profit'
airlines2015["Operating Profit"].hist()
plt.show()

# Boite à moustache de la variable 'Operating Profit'
airlines2015.boxplot(column=["Operating Profit"])

**Q** Etudier d'autres variables quantitatives.

#### 2.1.2 Variables qualitatives

In [ ]:
# Fréquence des valeurs pou 'Year'
freq_year = pd.crosstab(airlines["Year"], "freq")
print(freq_year)

# Diagramme en bâtons de 'Year'
freq_year.plot.bar()

# Camembert (à ne surtout pas faire car illisible!)
freq_year.plot.pie(subplots=True, figsize = (8, 8))

**Q** Etudier d'autres variables qualitatives.



### 2.2 Statistiques descriptives multivariées

#### 2.2.1 Statistiques quantitatives

In [ ]:
from pandas.plotting import scatter_matrix

# Scatter plot matrix des variables quantitatives sous forme d'histogramme
scatter_matrix(airlines2015[["Operating Profit","Operating Revenue","Net Result","Employees","Operating Expenses"]], alpha=0.2, figsize=(15, 15))
plt.show()

# Scatter plot matrix des variables quantitatives sous forme de densité
scatter_matrix(airlines2015[["Operating Profit","Operating Revenue","Net Result","Employees","Operating Expenses"]], alpha=0.2, figsize=(15, 15), diagonal='kde')
plt.show()

airlines2015.corr()

**Q** Commenter les liens 2 à 2 de ces variables.

#### 2.2.2 Statistiques qualitatives

In [ ]:
# Table de contingence entre 'Year' et 'Airline'
airline_year = pd.crosstab(airlines["Year"], airlines["Airline"], margins=True , normalize="index")

airline_year.plot.bar(stacked=True)

<hr style="height:2px" />

## 3 Analyse en composantes principales <a class="anchor" id="acp"></a>

Pour cette partie, on ne travaille qu'avec les variables quantitatives et en remplaçant les valeurs manquantes par la moyenne de la variable.

In [ ]:
# On sélectionne uniquement les variables quantitatives
airlines2015etude=airlines2015.select_dtypes(include=['float64'])

# On remplace les valeurs manquantes par la moyenne des variables
airlines2015etude = airlines2015etude.fillna(airlines2015etude.mean())
airlines2015etude.head(11)

In [ ]:
airlines2015quant=airlines2015etude.iloc[:,0:]
n = airlines2015quant.shape[0]
p = airlines2015quant.shape[1]
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
# Réduction des variables
X = scale(airlines2015quant)
# Vérification du centrage et de la réduction de notre dataframe
print(np.mean(X,axis=0))
print(np.std(X,axis=0,ddof=0))

# Calcul de l'ACP
pca_airlines = PCA()
C_airlines = pca_airlines.fit_transform(X)

liste=['Comp.'+str(i) for i in range(1,len(airlines2015etude.columns)+1)]
Cdataframe=pd.DataFrame(C_airlines, columns=liste)
display(Cdataframe)


### 3.1 Valeurs propres et choix du nombre de dimensions à garder

In [ ]:
# Eboulis des valeurs propres
plt.plot(pca_airlines.explained_variance_ratio_*100) # pour l'avoir en pourcentage
plt.show()

**Q** Combien de dimensions peut-on utiliser pour expliquer au moins 95% de l'information totale?

In [ ]:
# Calcul de la variane expliquée cumulée en pourcentage
print(pca_airlines.explained_variance_ratio_.cumsum()*100)

# Cumul de variance expliquée
plt.plot(np.arange(1,p+1),pca_airlines.explained_variance_ratio_.cumsum())
plt.title("Variance expliquée en fonction du nombre de composantes principales")
plt.ylabel("Variance expliquée cumulée")
plt.xlabel("Nombre de composantes principales")
plt.axhline(y = 0.95, linestyle = 'dashed')
plt.show()

### 3.2 Représentation des individus

In [ ]:
# Positionnement des individus dans le premier plan
limplot=[-4,13]
fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(limplot[0],limplot[1]) 
axes.set_ylim(limplot[0],limplot[1]) 

for i in range(n):
    plt.annotate(airlines2015quant.index[i],(C_airlines[i,0],C_airlines[i,1]), fontsize=7) # fontsize par défaut met des label trop gros
    
plt.plot([limplot[0],limplot[1]],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[limplot[0],limplot[1]],color='silver',linestyle='-',linewidth=1)

plt.show()

**Q** Quels sont les individus qui, selon vous, contribuent le plus à l'axe 1? A l'axe 2?

#### 3.2.1 Contribution des individus aux axes

In [ ]:
# Contributions aux axes
eigval = (n-1)/n*pca_airlines.explained_variance_
ctr = C_airlines**2/np.dot(np.array([np.ones(n)]).T,np.array([n*eigval]))

# étude de l'axe 1-2
contributionind = pd.DataFrame({'Index':airlines2015quant.index,'CTR_1':ctr[:,0],'CTR_2':ctr[:,1]})
contributionind = contributionind.set_index('Index')

## Pour voir tout le tableau des contributions (limité par défaut)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
#    # more options can be specified also
#    print(contributionplan)

# Individus dont la contribution à la 1ère composante est supérieure à 10%
print(contributionind.loc[contributionind['CTR_1']>0.1])

**Q** Quels sont les individus qui contribuent le plus à l'axe 2?

### 3.3 Représentation des variables

In [ ]:
# corrélation variables-facteurs
corvar = pca_airlines.components_.T*np.dot(np.array([np.ones(p)]).T,np.array([np.sqrt(eigval)]))

#### 3.3.1 Etude du plan 1-2

In [ ]:
# Cercle des corrélations
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)

# Affichage des étiquettes (noms des variables)
for j in range(p):
    plt.annotate(airlines2015quant.columns[j],(corvar[j,0],corvar[j,1]), fontsize=7)
    plt.arrow(0,0,corvar[j,0],corvar[j,1],color='black',length_includes_head=True, head_width=.03)
    
# Ajout des axes
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)

# Ajout du cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)

plt.show()

# Tableau des corrélations sur l'axe 1-2
correlation12 = pd.DataFrame({'Index':airlines2015quant.columns,'COR_1':corvar[:,0],'COR_2':corvar[:,1]})
correlation12 = correlation12.set_index('Index')

print()
print()
print(color.BOLD + color.BLUE + "Variables dont la corrélation avec la 1ère composante est supérieure à 50%:" + color.END)
display(correlation12.loc[correlation12['COR_1']>0.5])

print()
print()
print(color.BOLD + color.BLUE + "Variables dont la corrélation avec la 2ème composante est supérieure à 50%:" + color.END)
display(correlation12.loc[correlation12['COR_2']>0.5])

**Q** Quels sont, selon vous, les variables qui contribuent le plus à l'axe 1? à l'axe 2?

#### 3.3.2 Etude du plan 2-3

In [ ]:
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)
for j in range(p):
    plt.annotate(airlines2015quant.columns[j],(corvar[j,1],corvar[j,2]), fontsize=7)
    plt.arrow(0,0,corvar[j,1],corvar[j,2],color='black',length_includes_head=True, head_width=.03)
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)
plt.show()

# Tableau des corrélations sur l'axe 2-3
correlation23 = pd.DataFrame({'Index':airlines2015quant.columns,'COR_2':corvar[:,1],'COR_3':corvar[:,2]})
correlation23 = correlation23.set_index('Index')

print()
print()
print(color.BOLD + color.BLUE + "Variables dont la corrélation avec la 1ère composante est supérieure à 50%:" + color.END)
display(correlation23.loc[correlation23['COR_2']>0.5])

print()
print()
print(color.BOLD + color.BLUE + "Variables dont la corrélationa avec la 2ème composante est supérieure à 50%:" + color.END)
display(correlation23.loc[correlation23['COR_3']>0.5])

#### 3.3.3 Contribution des variables aux axes

In [ ]:
# Contribution variables pour la construction des axes
contribvar = corvar**2*np.dot(np.array([np.ones(p)]).T,np.array([1/(eigval)]))

# étude de l'axe 1-2
contributionvar = pd.DataFrame({'Index':airlines2015quant.columns,'CTR_1':contribvar[:,0],'CTR_2':contribvar[:,1]})
contributionvar = contributionvar.set_index('Index')

# Variables dont la contribution à la 1ère composante est supérieure à 5%
print(contributionvar.loc[contributionvar['CTR_1']>0.05])
print()
print()

# Variables dont la contribution à la 2ème composante est supérieure à 20%
print(contributionvar.loc[contributionvar['CTR_2']>0.2])

<hr style="height:2px" />

## 4. Classification <a class="anchor" id="cah"></a>

### 4.1 K-Means

In [ ]:
from sklearn.cluster  import  KMeans
from  sklearn.metrics  import confusion_matrix
###
# airline
#clust=KMeans(n_clusters=5)
Nb_clusters=5
clust_airlines=KMeans(n_clusters=Nb_clusters)
###
clust_airlines.fit(X)
groupes_kmeans=clust_airlines.labels_

centers = pd.DataFrame(clust_airlines.cluster_centers_, columns=airlines2015quant.columns.values.tolist())
display(centers)

In [ ]:
## Repésentation des individus dans les coordonnées de l'acp.
limplot=[-4,13]
fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(limplot[0],limplot[1]) 
axes.set_ylim(limplot[0],limplot[1])

## Repésentation des individus dans les coordonnées de l'acp.
plt.plot([limplot[0],limplot[1]],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[limplot[0],limplot[1]],color='silver',linestyle='-',linewidth=1)
plt.scatter(C_airlines[:,0], C_airlines[:,1], c=[couleur_plot[element] for element in groupes_kmeans]) 

## Représentation des centres des classes sur le même graphique (forme carré et plus gros)
couleur=list(set(groupes_kmeans))
Cgroup_kmeans_moy = Cdataframe.groupby(groupes_kmeans).mean()
plt.scatter(Cgroup_kmeans_moy.iloc[:,0], Cgroup_kmeans_moy.iloc[:,1], c=couleur_plot[0:Nb_clusters], marker='s',s=70)
plt.show()

In [ ]:
## Même représentation qu'au dessus, mais en 3D, dans une nouvelle fenêtre interactive
#%matplotlib qt
#from mpl_toolkits.mplot3d import Axes3D
#fig = plt.figure(1, figsize=(8, 6))
#ax = Axes3D(fig, elev=-150, azim=110)
#ax.scatter(C_airlines[:, 0], C_airlines[:, 1], C_airlines[:, 2], c=[couleur_plot[element] for element in groupes_kmeans],cmap=plt.cm.Paired)
#ax.scatter(Cgroup_kmeans_moy.iloc[:,0], Cgroup_kmeans_moy.iloc[:,1], c=couleur_plot[0:Nb_clusters], marker='s',s=70)
#ax.set_title("ACP: trois premieres composantes")
#ax.set_xlabel("Comp1")
#ax.w_xaxis.set_ticklabels([])
#ax.set_ylabel("Comp2")
#ax.w_yaxis.set_ticklabels([])
#ax.set_zlabel("Comp3")

### 4.2 Classification ascendante hiérarchique

#### 4.2.1 Méthode de Ward
Construction du dendrogramme

In [ ]:
%matplotlib inline
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
#générer la matrice des liens
Z_ward = linkage(X,method='ward',metric='euclidean')
#affichage du dendrogramme
fig, axes = plt.subplots(figsize=(15,15))
plt.title("CAH")
dendrogram(Z_ward,labels=airlines2015quant.index,orientation='left',color_threshold=0)
plt.show()

**Q** Combien de classes choisiriez-vous?

Pour comparer avec le "K-Means", nous choisissons de matérialiser la classification précédente avec 5 classes.

In [ ]:
# Matérialisation des 5 classes (hauteur t = 10)
fig, axes = plt.subplots(figsize=(15,15))
plt.title('CAH avec matérialisation des 5 classes')
dendrogram(Z_ward,labels=airlines2015quant.index,orientation='left',color_threshold=10)
plt.show()
# Découpage à la hauteur t = 10 ==> identifiants de 5 groupes obtenus
groupes_cah_ward= fcluster(Z_ward,t=10,criterion='distance')
print(groupes_cah_ward)
# Index triés des groupes
idg_ward= np.argsort(groupes_cah_ward)
# Affichage des observations et leurs groupes
print(pd.DataFrame(airlines2015quant.index[idg_ward],groupes_cah_ward[idg_ward]))

In [ ]:
limplot=[-4,13]
fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(limplot[0],limplot[1])
axes.set_ylim(limplot[0],limplot[1])

# Repésentation des individus dans les coordonnées de l'acp
#  (attention, les couleurs ne correspondent pas aux couleurs du dendrogramme) 
plt.plot([limplot[0],limplot[1]],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[limplot[0],limplot[1]],color='silver',linestyle='-',linewidth=1)
plt.scatter(C_airlines[:,0], C_airlines[:,1], c=[couleur_plot[element-1] for element in groupes_cah_ward])
couleur=list(set(groupes_cah_ward))
Cgroup_cah_ward_moy = Cdataframe.groupby(groupes_cah_ward).mean()
plt.scatter(Cgroup_cah_ward_moy.iloc[:,0], Cgroup_cah_ward_moy.iloc[:,1], c=couleur_plot[0:Nb_clusters], marker='s',s=70)
plt.show()

**Q** Quelles sont les différences/ressemblances avec la méthode des K-Means précédente?

Pour les prochains critères, nous allons utiliser 5 classes afin de comparer les différentes classifications. A vous de répondre à la même question précédente sur les différentes et ressemblances de classification.

#### 4.2.2 Critère du saut minimal

In [ ]:
#générer la matrice des liens
Z_single = linkage(X,method='single',metric='euclidean')
#matérialisation des 4 classes (hauteur t = 7)
fig, axes = plt.subplots(figsize=(15,15))
plt.title('CAH avec matérialisation des 5 classes')
dendrogram(Z_single,labels=airlines2015quant.index,orientation='left',color_threshold=3.6)
plt.show()
#découpage à la hauteur t = 3.7 ==> identifiants de 5 groupes obtenus
groupes_cah_single= fcluster(Z_single,t=3.7,criterion='distance')
print(groupes_cah_single)
#index triés des groupes
idg_single= np.argsort(groupes_cah_single)
#affichage des observations et leurs groupes
print(pd.DataFrame(airlines2015quant.index[idg_single],groupes_cah_single[idg_single]))

fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(limplot[0],limplot[1]) 
axes.set_ylim(limplot[0],limplot[1]) 

# Repésentation des individus dans les coordonnées de l'acp
#  (attention, les couleurs ne correspondent pas aux couleurs du dendrogramme) 
plt.plot([limplot[0],limplot[1]],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[limplot[0],limplot[1]],color='silver',linestyle='-',linewidth=1)
plt.scatter(C_airlines[:,0], C_airlines[:,1], c=[couleur_plot[element-1] for element in groupes_cah_single]) 
couleur=list(set(groupes_cah_single))
Cgroup_cah_single_moy = Cdataframe.groupby(groupes_cah_single).mean()
plt.scatter(Cgroup_cah_single_moy.iloc[:,0], Cgroup_cah_single_moy.iloc[:,1], c=couleur_plot[0:Nb_clusters], marker='s',s=70)
plt.show()

*Différence/ressemblance entre méthodes*

*Réponse:*   

#### 4.2.3 Critère du diamètre

In [ ]:
#générer la matrice des liens
Z_complete = linkage(X,method='complete',metric='euclidean')
##affichage du dendrogramme
#fig, axes = plt.subplots(figsize=(15,15))
#plt.title("CAH")
#dendrogram(Z,labels=airlines2015quant.index,orientation='left',color_threshold=0)
#plt.show()
#matérialisation des 4 classes (hauteur t = 7)
fig, axes = plt.subplots(figsize=(15,15))
plt.title('CAH avec matérialisation des 5 classes')
dendrogram(Z_complete,labels=airlines2015quant.index,orientation='left',color_threshold=7.4)
plt.show()
#découpage à la hauteur t = 7 ==> identifiants de 5 groupes obtenus
groupes_cah_complete= fcluster(Z_complete,t=7.4,criterion='distance')
print(groupes_cah_complete)
#index triés des groupes
idg_complete= np.argsort(groupes_cah_complete)
#affichage des observations et leurs groupes
print(pd.DataFrame(airlines2015quant.index[idg_complete],groupes_cah_complete[idg_complete]))

fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(limplot[0],limplot[1]) 
axes.set_ylim(limplot[0],limplot[1]) 
# Repésentation des individus dans les coordonnées de l'acp
#  (attention, les couleurs ne correspondent pas aux couleurs du dendrogramme) 
plt.plot([limplot[0],limplot[1]],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[limplot[0],limplot[1]],color='silver',linestyle='-',linewidth=1)
plt.scatter(C_airlines[:,0], C_airlines[:,1], c=[couleur_plot[element-1] for element in groupes_cah_complete]) 
couleur=list(set(groupes_cah_complete))
Cgroup_cah_complete_moy = Cdataframe.groupby(groupes_cah_complete).mean()
plt.scatter(Cgroup_cah_complete_moy.iloc[:,0], Cgroup_cah_complete_moy.iloc[:,1], c=couleur_plot[0:Nb_clusters], marker='s',s=70)
plt.show()

*Différence/ressemblance entre méthodes*

*Réponse:*   

#### 4.2.4 Critère de la moyenne

In [ ]:
#générer la matrice des liens
Z_average = linkage(X,method='average',metric='euclidean')
##affichage du dendrogramme
#fig, axes = plt.subplots(figsize=(15,15))
#plt.title("CAH")
#dendrogram(Z,labels=airlines2015quant.index,orientation='left',color_threshold=0)
#plt.show()
#matérialisation des 4 classes (hauteur t = 5)
fig, axes = plt.subplots(figsize=(15,15))
plt.title('CAH avec matérialisation des 5 classes')
dendrogram(Z_average,labels=airlines2015quant.index,orientation='left',color_threshold=5.4)
plt.show()
#découpage à la hauteur t = 5 ==> identifiants de 5 groupes obtenus
groupes_cah_average= fcluster(Z_average,t=5.4,criterion='distance')
print(groupes_cah_average)
#index triés des groupes
idg_average= np.argsort(groupes_cah_average)
#affichage des observations et leurs groupes
print(pd.DataFrame(airlines2015quant.index[idg_average],groupes_cah_average[idg_average]))

fig, axes = plt.subplots(figsize=(12,12))
axes.set_xlim(limplot[0],limplot[1]) 
axes.set_ylim(limplot[0],limplot[1]) 
# Repésentation des individus dans les coordonnées de l'acp
#  (attention, les couleurs ne correspondent pas aux couleurs du dendrogramme) 
plt.plot([limplot[0],limplot[1]],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[limplot[0],limplot[1]],color='silver',linestyle='-',linewidth=1)
plt.scatter(C_airlines[:,0], C_airlines[:,1], c=[couleur_plot[element-1] for element in groupes_cah_average]) 
couleur=list(set(groupes_cah_average))
Cgroup_cah_average_moy = Cdataframe.groupby(groupes_cah_average).mean()
plt.scatter(Cgroup_cah_average_moy.iloc[:,0], Cgroup_cah_average_moy.iloc[:,1], c=couleur_plot[0:Nb_clusters], marker='s',s=70)
plt.show()

*Différence/ressemblance entre méthodes*

*Réponse:*   

<br>       

**Q** Au vu de toutes les classifications effectuées et de l'ACP, quelle autre analyse pourrait-on faire?

**Q (bonus)** Créer une fonction qui permet de fournir en sortie la classification (graphiques, groupe,...) avec comme paramètre d'entrée: la méthode (ward, single...) et le nombre de classe.

### 4.3 Interprétation des classes

Choix de la méthode de classification pour l'interprétation

In [ ]:
## Choix des classes pour les individus en fonction de la méthode
########
#groupes=groupes_kmeans
groupes=groupes_cah_ward
#groupes=groupes_cah_single
#groupes=groupes_cah_complete
#groupes=groupes_cah_average

groupes_moy = Cdataframe.groupby(groupes).mean()

#### 4.3.1 Par les variables

In [ ]:
# Moyenne par variable
m = airlines2015quant.mean()
nbvar=len(m)
# Ecart-type par variable
sd = airlines2015quant.std()

# Dataframe conditionnellement aux groupes
airlines2015group = airlines2015quant.groupby(groupes)

# Effectifs conditionnels
airlines2015group_n= airlines2015group.size()
#print(airlines2015group_n)
N=sum(airlines2015group_n)

# Moyennes conditionnelles
airlines2015group_moy= airlines2015group.mean()
# Ecart-type conditionnelles
airlines2015group_std= airlines2015group.std()

# Calcul du V-test
a=(airlines2015group_moy-m)/sd
b=np.dot(np.array([((airlines2015group_n*(N-1))/(N-airlines2015group_n))**(0.5)]).T,
         np.full(shape=(1,nbvar),fill_value=1))
vtest=a*b

from scipy.stats import norm
def pvaleur(X):
    return 2*norm.cdf(-abs(X))

for i in range(vtest.shape[0]):
    df = pd.DataFrame({'vtest': vtest.iloc[i,:], 'moyenne-conditionnelle': airlines2015group_moy.iloc[i,:],
                   'moyenne totale': m, 'variance conditionnelle': airlines2015group_std.iloc[i,:],
                       'variance totale': sd,'pvaleur' : pvaleur(vtest.iloc[i,:])},
                      index = vtest.columns)
    dfselection = df.loc[df['pvaleur']<0.05,:]
#    dfselection = df
    print(color.BOLD + color.BLUE + "Classe " + str(i+1) + " (n=" + str(airlines2015group_n.iloc[i]) + ")"+ color.END)
    display(dfselection.sort_values(by = 'pvaleur'))
    print()
    print()


**Q** Interpréter les résultats.

#### 4.3.2 Par les composantes principales

In [ ]:
# Moyenne par composantes principales
Cm = Cdataframe.mean()
nbvar=len(Cm)
# Ecart-type par composantes principales
Csd = Cdataframe.std()

# Dataframe conditionnellement aux groupes
Cgroup = Cdataframe.groupby(groupes)

# Effectifs conditionnels
Cgroup_n= Cgroup.size()
N=sum(Cgroup_n)

# Moyennes conditionnelles
Cgroup_moy= Cgroup.mean()
# Ecart-type conditionnelles
Cgroup_std= Cgroup.std()

# Calcul du V-test
Ca=(Cgroup_moy-Cm)/Csd
Cb=np.dot(np.array([((Cgroup_n*(N-1))/(N-Cgroup_n))**(0.5)]).T,
         np.full(shape=(1,nbvar),fill_value=1))
Cvtest=Ca*Cb

for i in range(Cvtest.shape[0]):
    df = pd.DataFrame({'vtest': Cvtest.iloc[i,:], 'moyenne-conditionnelle': Cgroup_moy.iloc[0,:],
                   'moyenne totale': Cm,'variance-conditionnelle': Cgroup_std.iloc[0,:],
                   'variance totale': Csd, 'pvaleur' : pvaleur(Cvtest.iloc[i,:])},
                      index = Cvtest.columns)
    dfselection = df.loc[df['pvaleur']<0.05,:]
    print(color.BOLD + color.BLUE + "Classe " + str(i+1) + " (n=" + str(Cgroup_n.iloc[i]) + ")"+ color.END)
    #print(i)
    display(dfselection.sort_values(by = 'pvaleur'))
    print()
    print()


**Q** Interpréter les résultats.

#### 4.3.3 Par les individus

In [ ]:
# Calcul des parangons par classe
# Calcul des individus les plus éloignés des centres des autres classes par classe 
#      (on prend le minimum des distances avec tous les autres centres de classes)

import scipy
tempo = scipy.spatial.distance.cdist(Cdataframe, groupes_moy, metric='euclidean')
tempo=pd.DataFrame(tempo, index=airlines2015etude.index)

i=0
parangon_ind = []
distance_ind = []
for valeur in Cgroup.indices.values():
    tempobis=tempo.iloc[valeur,:]
    parangon=tempobis.iloc[:,i]
    del tempobis[i]
    tempobisbis=tempobis.iloc[:,:]
    distance=np.transpose(tempobisbis).min()
    parangon_ind.append([parangon.sort_values()])
    distance_ind.append([distance.sort_values(ascending=False)])
    i=i+1

print(color.BOLD + color.BLUE + "PARANGON" + color.END)
for j in Cgroup.indices.keys():
    print(color.BOLD + color.BLUE + "Classe " + str(j) + " (n=" + str(Cgroup_n.iloc[j-1]) + ")"+ color.END)
    parangon_groupes=pd.DataFrame(parangon_ind[j-1],index=["Distance"])
    display(parangon_groupes)
    
print()
print(color.BOLD + color.BLUE + "DISTANCE MAX" + color.END)
for k in Cgroup.indices.keys():
    print(color.BOLD + color.BLUE + "Classe " + str(k) + " (n=" + str(Cgroup_n.iloc[k-1]) + ")"+ color.END)
    distance_groupes=pd.DataFrame(distance_ind[k-1],index=["Distance"])
    display(distance_groupes)

**Q** Interpréter les résultats.